We install here the required dependencies

In [21]:
!python -m pip install kuzu

In [22]:
import kuzu

db = kuzu.Database('./kuzudb_elt')
conn = kuzu.Connection(db)

Let's drop the old edges and then nodes (if they exist)

In [23]:
SHOW_EDGE_TABLES = """
CALL SHOW_TABLES() WHERE type='REL' RETURN *;
"""
result_set = conn.execute(SHOW_EDGE_TABLES)
while result_set.has_next():
    table_name = result_set.get_next()[0]
    print(f"DROPPING {table_name}")
    conn.execute(f"DROP TABLE {table_name};")

result_set.close()

SHOW_NODE_TABLES = """
CALL SHOW_TABLES() WHERE type='NODE' RETURN *;
"""
result_set = conn.execute(SHOW_NODE_TABLES)
while result_set.has_next():
    table_name = result_set.get_next()[0]
    print(f"DROPPING {table_name}")
    conn.execute(f"DROP TABLE {table_name};")

result_set.close()

DROPPING OMDB_BUSINESS_CATEGORY
DROPPING OMDB_BUSINESS_JOB
DROPPING OMDB_BUSINESS_MOVIE
DROPPING OMDB_BUSINESS_MOVIESERIES
DROPPING OMDB_BUSINESS_CAST
DROPPING OMDB_BUSINESS_CHARACTER
DROPPING OMDB_BUSINESS_EPISODE
DROPPING OMDB_BUSINESS_IMAGE


Now we load the nodes.

In [24]:
with open('dml_statements/elt_kuzudb_node_schema.sql') as f:
    creation_lines = [line for line in f]

with open('dml_statements/elt_kuzudb_node_import.sql') as f:
    import_lines = [line for line in f]
    

#do not forget the primary keys for seeds

    
for line in creation_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

for line in import_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CAST(cast_id STRING, roles STRING[], PRIMARY KEY (cast_id));

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CATEGORY(category_id INT64, parent_id INT64, root_id INT64, names STRUCT(name STRING, language_iso_639_1 STRING)[], PRIMARY KEY (category_id));

Executing statement CREATE NODE TABLE OMDB_BUSINESS_CHARACTER(id INT64, name STRING, PRIMARY KEY (id));

Executing statement CREATE NODE TABLE OMDB_BUSINESS_EPISODE(episode_id INT64, series_id INT64, season_id INT64, date DATE, content_type STRING, runtime INT64, revenue INT64, budget INT64, homepage STRING, vote_average FLOAT, votes_count INT64, last_update TIMESTAMP, countries STRING[],  abstracts STRUCT(abstract STRING, language_iso_639_1 STRING)[], content_links STRUCT(source STRING, key STRING, language_iso_639_1 STRING)[], refs STRUCT(type STRING, referenced_id INT64)[], trailers STRUCT(source STRING, key STRING, language_iso_639_1 STRING)[], aliasings STRUCT(name STRING, la

RuntimeError: Conversion exception: Cast failed. {"(Genre,de)","(Genre,fr)","(Gatunek,pl)","(Género,es)","(Genre,en)"} is not in STRUCT(name STRING, language_iso_639_1 STRING)[] range.

Now we load the relationships.

In [5]:
with open('dml_statements/elt_kuzudb_rel_schema.sql') as f:
    creation_lines = [line for line in f]

with open('dml_statements/elt_kuzudb_rel_import.sql') as f:
    import_lines = [line for line in f]
    

#do not forget the primary keys for seeds

    
for line in creation_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

for line in import_lines:
    print(f"Executing statement {line}")
    conn.execute(line)

Executing statement CREATE REL TABLE HAS_ATTRIBUTE(FROM IMDB_TITLE_AKAS TO IMDB_ATTRIBUTE);

Executing statement CREATE REL TABLE HAS_CHARACTER(FROM IMDB_TITLE_PRINCIPAL TO IMDB_CHARACTER);

Executing statement CREATE REL TABLE HAS_DIRECTOR(FROM IMDB_TITLE_BASICS TO IMDB_NAME_BASICS);

Executing statement CREATE REL TABLE HAS_DISTRIBUTION_TYPE(FROM IMDB_TITLE_AKAS TO IMDB_DISTRIBUTION_TYPE);

Executing statement CREATE REL TABLE HAS_LANGUAGE(FROM IMDB_TITLE_AKAS TO IMDB_LANGUAGE_CODE);

Executing statement CREATE REL TABLE HAS_GENRE(FROM IMDB_TITLE_BASICS TO IMDB_GENRE);

Executing statement CREATE REL TABLE HAS_JOB_CATEGORY(FROM IMDB_TITLE_PRINCIPAL TO IMDB_JOB_CATEGORY);

Executing statement CREATE REL TABLE IS_KNOWN_FOR(FROM IMDB_NAME_BASICS TO IMDB_TITLE_BASICS);

Executing statement CREATE REL TABLE HAS_NAME(FROM IMDB_TITLE_PRINCIPAL TO IMDB_NAME_BASICS);

Executing statement CREATE REL TABLE HAS_ORIGINAL_TITLE(FROM IMDB_TITLE_AKAS TO IMDB_TITLE_BASICS);

Executing statement CREAT